## [0단계] 사용할 라이브러리와 패키지 불러오기

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## [1단계] 예측 모델 선정

In [11]:
df = pd.read_csv("../data/merged/traffic_weather_with_cluster.csv")
df['datetime'] = pd.to_datetime(df['datetime'])

df['date'] = df['datetime'].dt.floor('D')
df['hour'] = df['datetime'].dt.hour
df["is_rain"] = (df["precipitation"] > 0).astype(int)

df = df.sort_values(["intersection", "datetime"])

df["lag1"] = df.groupby("intersection")["traffic_volume"].shift(1)
df["lag2"] = df.groupby("intersection")["traffic_volume"].shift(2)
df["lag3"] = df.groupby("intersection")["traffic_volume"].shift(3)

df["roll3"] = df.groupby("intersection")["traffic_volume"].rolling(3).mean().reset_index(0,drop=True)
df["roll6"] = df.groupby("intersection")["traffic_volume"].rolling(6).mean().reset_index(0,drop=True)

df = df.dropna()

# =====================================================
# 2. Train / Test 분리 (최근 30일)
# =====================================================
test_size_days = 30
split_date = df["date"].max() - pd.Timedelta(days=test_size_days)

train = df[df["date"] <= split_date]
test  = df[df["date"] > split_date]

drop_cols = ["traffic_volume", "intersection", "datetime", "date"]

X_train = train.drop(columns=drop_cols)
y_train = train["traffic_volume"]

X_test = test.drop(columns=drop_cols)
y_test = test["traffic_volume"]

# =====================================================
# 3. 후보 1) Linear Regression
# =====================================================

lr = LinearRegression()
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)

rmse_lr = mean_squared_error(y_test, pred_lr) ** 0.5
mae_lr = mean_absolute_error(y_test, pred_lr)

print("Baseline (LR) RMSE:", rmse_lr)
print("Baseline (LR) MAE:", mae_lr)

# =====================================================
# 4. 후보 2) RandomForest
# =====================================================
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

rmse_rf = mean_squared_error(y_test, pred_rf) ** 0.5
mae_rf = mean_absolute_error(y_test, pred_rf)

print("RF RMSE:", rmse_rf)
print("RF MAE:", mae_rf)

# =====================================================
# 5. 후보 3) XGBoost
# =====================================================
xgb = XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)

rmse_xgb = mean_squared_error(y_test, pred_xgb) ** 0.5
mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGB RMSE:", rmse_xgb)
print("XGB MAE:", mae_xgb)


Baseline (LR) RMSE: 7.527587408906e-12
Baseline (LR) MAE: 5.774439071210237e-12
RF RMSE: 96.32553394208085
RF MAE: 59.15137369635231
XGB RMSE: 70.97879786312072
XGB MAE: 41.228782653808594


#### 테스트 결과, 현재 데이터 구조에서는 XGBoost가 적합하다고 판단 -> XGBoost로 고정

## [2단계] 테스트 기간(7, 30, 60, 90일)별 성능 비교

In [14]:
# 임의로 특정 교차로 선택
target = "2공단3"
df = df[df["intersection"] == target].copy()


df["roll3"] = (
    df.groupby("intersection")["traffic_volume"]
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
)

df["roll6"] = (
    df.groupby("intersection")["traffic_volume"]
      .rolling(6).mean()
      .reset_index(level=0, drop=True)
)

# 비교할 테스트 기간 목록
test_days_list = [7, 30, 60, 90]
results = []

for test_days in test_days_list:

    split_date = df["date"].max() - pd.Timedelta(days=test_days)

    train = df[df["date"] <= split_date]
    test  = df[df["date"] > split_date]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]

    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    # XGBoost 모델로 실행
    xgb = XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.05,
        random_state=42,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist"
    )

    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_test)

    rmse_xgb = (mean_squared_error(y_test, pred_xgb)) ** 0.5
    mae_xgb = mean_absolute_error(y_test, pred_xgb)

    # 결과 저장
    results.append({
        "test_days": test_days,
        "XGB_RMSE": rmse_xgb,
        "XGB_MAE": mae_xgb
    })


# 결과 표 출력
results_df = pd.DataFrame(results)
print(results_df)


   test_days    XGB_RMSE     XGB_MAE
0          7  200.323254  114.959496
1         30  168.213882   96.413803
2         60  210.644936  118.021614
3         90  203.655116  114.838440


#### 테스트 결과, 테스트 기간은 30일이 가장 성능이 우수

## [3단계] 클러스터별 모델 예측

In [ ]:
df["lag1"]  = df.groupby("intersection")["traffic_volume"].shift(1)
df["lag2"]  = df.groupby("intersection")["traffic_volume"].shift(2)
df["lag3"]  = df.groupby("intersection")["traffic_volume"].shift(3)

df["lag24"] = df.groupby("intersection")["traffic_volume"].shift(24)
df["lag48"] = df.groupby("intersection")["traffic_volume"].shift(48)
df['lag72'] = df.groupby("intersection")['traffic_volume'].shift(72)

df["roll3"]  = df.groupby("intersection")["traffic_volume"].rolling(3).mean().reset_index(0,drop=True)
df["roll6"]  = df.groupby("intersection")["traffic_volume"].rolling(6).mean().reset_index(0,drop=True)
df["roll24"] = df.groupby("intersection")["traffic_volume"].rolling(24).mean().reset_index(0,drop=True)
df['roll48'] = df.groupby("intersection")['traffic_volume'].rolling(48).mean().reset_index(level=0, drop=True)


# 시간 파생변수
df["dayofweek"] = df["datetime"].dt.dayofweek

# Cyclic Encoding
df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)
df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)

df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)


# 클러스터별로 분리
df_c0 = df[df["cluster"] == 0].copy()   # 중/저혼잡형
df_c1 = df[df["cluster"] == 1].copy()   # 상시혼잡형

print("Cluster 0 데이터 수:", len(df_c0))
print("Cluster 1 데이터 수:", len(df_c1))


# Train/Test 분리 함수
def split_data(df, test_days=30):
    split_date = df["date"].max() - pd.Timedelta(days=test_days)
    
    train = df[df["date"] <= split_date]
    test  = df[df["date"] > split_date]

    drop_cols = ["traffic_volume", "intersection", "datetime", "date"]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]

    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    return X_train, y_train, X_test, y_test


# 모델 + 평가 함수
def train_xgb(X_train, y_train):
    model = XGBRegressor(
        n_estimators=400,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=1.0,
        gamma=1,
        min_child_weight=2,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model


def evaluate(model, X_test, y_test):
    pred = model.predict(X_test)
    rmse = (mean_squared_error(y_test, pred))**0.5
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    return rmse, mae, r2


# 클러스터별 예측 실행

# --- Cluster 0 ---
X_train0, y_train0, X_test0, y_test0 = split_data(df_c0)
model0 = train_xgb(X_train0, y_train0)
rmse0, mae0, r20 = evaluate(model0, X_test0, y_test0)

# --- Cluster 1 ---
X_train1, y_train1, X_test1, y_test1 = split_data(df_c1)
model1 = train_xgb(X_train1, y_train1)
rmse1, mae1, r21 = evaluate(model1, X_test1, y_test1)

print("\n==============================")
print("📌 클러스터별 예측 성능 (최종)")
print("==============================")

print(f"[Cluster 0 - 중·저혼잡형]  RMSE: {rmse0:.2f} | MAE: {mae0:.2f} | R²: {r20:.4f}")
print(f"[Cluster 1 - 상시혼잡형]  RMSE: {rmse1:.2f} | MAE: {mae1:.2f} | R²: {r21:.4f}")

print("\n🎯 클러스터 기반 예측 완료!")


Cluster 0 데이터 수: 336378
Cluster 1 데이터 수: 253734

📌 클러스터별 예측 성능 (최종)
[Cluster 0 - 중·저혼잡형]  RMSE: 41.41 | MAE: 20.07 | R²: 0.9984
[Cluster 1 - 상시혼잡형]  RMSE: 69.20 | MAE: 34.82 | R²: 0.9983

🎯 클러스터 기반 예측 완료!
